In [1]:
%run ../config/init.py
PIPELINE_JSON = os.path.join(CONFIG, 'gcp', 'pipeline.json')

In [2]:
with open(PIPELINE_JSON) as fin:
    pipeline = json.load(fin)
print(json.dumps(pipeline, indent=2))

{
  "actions": [
    {
      "commands": [
        "/bin/sh",
        "-c",
        "mkdir -v /data/${SAMPLE}_${N}_${CPUs} > /data/pipeline.log 2>&1"
      ],
      "imageUri": "google/cloud-sdk:slim",
      "mounts": [
        {
          "disk": "gcloud-shared",
          "path": "/data"
        }
      ]
    },
    {
      "commands": [
        "/bin/sh",
        "-c",
        "gsutil cp gs://cbb-research-dl-blastdb/BOTO_CONFIG /data/.boto >> /data/pipeline.log 2>&1"
      ],
      "imageUri": "google/cloud-sdk:slim",
      "mounts": [
        {
          "disk": "gcloud-shared",
          "path": "/data"
        }
      ]
    },
    {
      "commands": [
        "/bin/sh",
        "-c",
        "gsutil -m -o 'GSUtil:parallel_composite_upload_threshold=150M' -o 'GSUtil:parallel_process_count=16' -o 'GSUtil:parallel_thread_count=16' cp gs://cbb-research-dl-blastdb/* /data/ >> /data/pipeline.log 2>&1"
      ],
      "imageUri": "google/cloud-sdk:slim",
      "mounts": [
        {
    

In [26]:
pipeline['resources']['virtualMachine']['machineType'] = 'n1-standard-64'
print(json.dumps(pipeline['resources']['virtualMachine'], indent=2))

{
  "bootDiskSizeGb": 60,
  "bootImage": "projects/cos-cloud/global/images/family/cos-stable",
  "disks": [
    {
      "name": "gcloud-shared",
      "sizeGb": "1500",
      "type": "local-ssd"
    }
  ],
  "machineType": "n1-standard-64",
  "preemptible": false,
  "serviceAccount": {
    "email": "default",
    "scopes": [
      "https://www.googleapis.com/auth/cloud-platform"
    ]
  }
}


In [8]:
with open('pipeline.json', 'w') as f:
        f.write(json.dumps(pipeline, indent=2))

In [7]:
with gzip.GzipFile('pipeline.json.gz', 'w') as fout:   # 4. gzip
    fout.write(json.dumps(pipeline, indent=2).encode('utf-8'))                  

In [4]:
!pwd

/panfs/pan1.be-md.ncbi.nlm.nih.gov/alt_splicing/cloud-transcriptome-annotation/notebooks


In [11]:
with gzip.GzipFile('pipeline.json.gz', 'r') as fin:    # 4. gzip
    json_bytes = fin.read()
    l = json.loads(json_bytes.decode('utf-8'))

In [12]:
l

{'actions': [{'commands': ['/bin/sh',
    '-c',
    'mkdir -v /data/${SAMPLE}_${N}_${CPUs} > /data/pipeline.log 2>&1'],
   'imageUri': 'google/cloud-sdk:slim',
   'mounts': [{'disk': 'gcloud-shared', 'path': '/data'}]},
  {'commands': ['/bin/sh',
    '-c',
    'gsutil cp gs://cbb-research-dl-blastdb/BOTO_CONFIG /data/.boto >> /data/pipeline.log 2>&1'],
   'imageUri': 'google/cloud-sdk:slim',
   'mounts': [{'disk': 'gcloud-shared', 'path': '/data'}]},
  {'commands': ['/bin/sh',
    '-c',
    "gsutil -m -o 'GSUtil:parallel_composite_upload_threshold=150M' -o 'GSUtil:parallel_process_count=16' -o 'GSUtil:parallel_thread_count=16' cp gs://cbb-research-dl-blastdb/* /data/ >> /data/pipeline.log 2>&1"],
   'imageUri': 'google/cloud-sdk:slim',
   'mounts': [{'disk': 'gcloud-shared', 'path': '/data'}]},
  {'commands': ['/bin/sh',
    '-c',
    'gsutil -m -q cp gs://${INBUCKET}/${SAMPLE}.fa /data/ >> /data/pipeline.log 2>&1'],
   'imageUri': 'google/cloud-sdk:slim',
   'mounts': [{'disk': 'gclou